Example of a wrapper for turning various user inputs into astropy SkyCoord objects following 

http://docs.astropy.org/en/stable/coordinates/

Also comparing with *astroquery.utils.parse_coordinates*, and a code it uses internally, *astroquery.vo_conesearch.core._validate_coords()*

    - First off, parse_coordinates() takes only one parameter, and it cannot be a list or tuple. So it must be either a string or a coordinates object. No lists of ras and decs in any format. 
    - Secondly, _validate_coords() doesn't always work consistently, as shown below.


In [1]:
from navo_utils import utils
from astroquery.utils import parse_coordinates
from astroquery.vo_conesearch import core

### two floats

In [2]:
utils.parse_coords([10.625, 41.2])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [3]:
utils.parse_coords([[10.625, 41.2],[10.625, 41.2],[10.625, 41.2]])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [4]:
core._validate_coord([10.625, 41.2])

(10.625, 41.2)

In [5]:
## Doesn't work with a list
# core._validate_coord([[10.625, 41.2],[10.625, 41.2],[10.625, 41.2]])

In [6]:
## Doesn't work with list or tuple for a single object. Only the underlying _validate_coord does this.
#parse_coordinates((10.625, 41.2))

### two strings decimals

In [7]:
utils.parse_coords(['10.625', '41.2'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [8]:
utils.parse_coords('10.625 41.2')

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [9]:
core._validate_coord(['10.625', '41.2'])

(10.625, 41.2)

In [10]:
## Doesn't take two
#parse_coordinates(['10.625', '41.2'])

### two strings HHhMMmSSs

In [11]:
utils.parse_coords(['00h42m30s', '+41d12m00s'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [12]:
utils.parse_coords([['00h42m30s', '+41d12m00s'],['00h42m30s', '+41d12m00s']])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [13]:
core._validate_coord(['00h42m30s', '+41d12m00s'])

(10.624999999999998, 41.2)

In [14]:
## Doesn't take two
#parse_coordinates(('00h42m30s', '+41d12m00s'))

### two strings, HHhMM.Mm

In [15]:
utils.parse_coords(['00h42.5m', '+41d12m'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [16]:
utils.parse_coords([['00h42.5m', '+41d12m'],['00h42.5m', '+41d12m']])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [17]:
core._validate_coord(['00h42.5m', '+41d12m'])

(10.625, 41.2)

### two strings HH MM SS

In [18]:
utils.parse_coords(['00 42 30', '+41 12 00'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [19]:
utils.parse_coords([['00 42 30', '+41 12 00'],['00 42 30', '+41 12 00']])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

# But this comes out differently!

    def _validate_coord(coordinates):
        """Validate coordinates and return them as ICRS RA and DEC in deg."""
        if isinstance(coordinates, (list, tuple)) and len(coordinates) == 2:
            icrscoord = ICRS(Longitude(coordinates[0], unit=u.degree),
                             Latitude(coordinates[1], unit=u.degree))
        else:
            c = commons.parse_coordinates(coordinates)
            if isinstance(c, SkyCoord):
                icrscoord = c.transform_to(ICRS).frame
            elif isinstance(c, BaseCoordinateFrame):
                icrscoord = c.transform_to(ICRS)
            else:  # Assume already ICRS
                icrscoord = c
    
        return icrscoord.ra.degree, icrscoord.dec.degree

So when it's given a list/tuple, then it assumes degrees. The parse_coordinates() call that uses this internally doesn't have that problem. 

In [20]:
print(parse_coordinates('00 42 30 +41 12 00'))
print(core._validate_coord(['00 42 30', '+41 12 00']))

<SkyCoord (ICRS): (ra, dec) in deg
    (10.626, 41.2001)>
(0.7083333333333333, 41.2)


### two strings HH:MM.M

In [21]:
utils.parse_coords(['00:42.30','+41:12.00'])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.575, 41.2)>]

In [22]:
utils.parse_coords([['00:42.30','+41:12.00'],['00:42.30','+41:12.00']])

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.575, 41.2)>, <SkyCoord (ICRS): (ra, dec) in deg
     (10.575, 41.2)>]

# But this comes out differently!

In [23]:
core._validate_coord(['00:42.30','+41:12.00'])

(0.705, 41.2)

### one string HH MM SS

In [24]:
utils.parse_coords('00 42 30 +41 12 00')

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [25]:
parse_coordinates('00 42 30 +41 12 00')

<SkyCoord (ICRS): (ra, dec) in deg
    (10.626, 41.2001)>

In [26]:
core._validate_coord('00 42 30 +41 12 00')

(10.626, 41.2001)

### one string DD.ddd DD.ddd

In [27]:
utils.parse_coords('10.625 41.2')

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [44]:
parse_coordinates('10.626 41.2001').to_string('hmsdms')

'00h42m30.288s +41d12m00.432s'

In [42]:
core._validate_coord('10.625 41.2')

AttributeError: 'tuple' object has no attribute 'to_string'

### one string HHhMMmSSs HHhMMmSSs


In [30]:
utils.parse_coords('00h42m30s +41d12m00s')

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [31]:
parse_coordinates('00h42m30s +41d12m00s')

/Users/tjaffe/anaconda3/lib/python3.6/site-packages/astroquery/utils/commons.py:162: UserWarning: Coordinate string is being interpreted as an ICRS coordinate.
  warnings.warn("Coordinate string is being interpreted as an "


<SkyCoord (ICRS): (ra, dec) in deg
    (10.625, 41.2)>

In [32]:
core._validate_coord('00h42.5m +41d12m')

/Users/tjaffe/anaconda3/lib/python3.6/site-packages/astroquery/utils/commons.py:162: UserWarning: Coordinate string is being interpreted as an ICRS coordinate.
  warnings.warn("Coordinate string is being interpreted as an "


(10.625, 41.2)

### one string HHhMM.Mm HHhMM.Mm

In [33]:
utils.parse_coords('00h42.5m +41d12m')

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.625, 41.2)>]

In [34]:
parse_coordinates('00h42.5m +41d12m')

/Users/tjaffe/anaconda3/lib/python3.6/site-packages/astroquery/utils/commons.py:162: UserWarning: Coordinate string is being interpreted as an ICRS coordinate.
  warnings.warn("Coordinate string is being interpreted as an "


<SkyCoord (ICRS): (ra, dec) in deg
    (10.625, 41.2)>

In [35]:
core._validate_coord('00h42.5m +41d12m')

/Users/tjaffe/anaconda3/lib/python3.6/site-packages/astroquery/utils/commons.py:162: UserWarning: Coordinate string is being interpreted as an ICRS coordinate.
  warnings.warn("Coordinate string is being interpreted as an "


(10.625, 41.2)

### one string HH:MM.Mm HH:MM.Mm

In [36]:
utils.parse_coords('00:42.30 +41:12.00')

[<SkyCoord (ICRS): (ra, dec) in deg
     (10.575, 41.2)>]

In [37]:
parse_coordinates('00:42.30 +41:12.00')

<SkyCoord (ICRS): (ra, dec) in deg
    (10.5756, 41.20004)>

In [38]:
core._validate_coord('00:42.30 +41:12.00')

(10.5756, 41.20004)

In [39]:
utils.parse_coords()

TypeError: parse_coords() missing 1 required positional argument: 'incoords'

In [ ]:
parse_coordinates()

In [ ]:
core._validate_coord()

In [ ]:
utils.parse_coords()

In [ ]:
parse_coordinates()

In [ ]:
core._validate_coord()